In [1]:


!pip install duckdb
import duckdb
import pandas as pd


try:
   
    df = pd.read_csv('/kaggle/input/journal-entries/journal_entry_anomalies.csv')
    
   
    print("--- Data Loading ---")
    print("Successfully loaded CSV. Columns are:", df.columns.tolist())
    
    # Register the pandas DataFrame as a virtual table in DuckDB
    duckdb.register("journal_entries", df)
    
except FileNotFoundError:
    print("--- Data Loading Failed ---")
    print("Error: The file '/kaggle/input/journal-entries/journal_entry_anomalies.csv' was not found.")
    print("Please ensure the file path is correct.")
   
    exit()

print("\n" + "="*50 + "\n") # Separator for clarity



queries_to_run = [
    {
        "title": "Query 1: Top 10 Highest Value Transactions",
        "sql": """
            SELECT *
            FROM journal_entries
            ORDER BY Amount DESC
            LIMIT 10;
        """
    },
    {
        "title": "Query 2: Transactions Posted on a Weekend",
        "sql": """
            SELECT *
            FROM journal_entries
            WHERE DAYOFWEEK(strptime(Date, '%d-%m-%Y')) IN (0, 6); -- Sunday=0, Saturday=6
        """
    },
    {
        "title": "Query 3: Top 10 Users by Number of Entries Made",
        "sql": """
            SELECT
    Created_By,
    COUNT(*) AS EntryCount
FROM
    journal_entries
GROUP BY
    Created_By
ORDER BY
    EntryCount DESC
LIMIT 10;
        """
    },
    {
        "title": "Query 4: Frequently Recurring Transaction Amounts (Potential Automation/Error)",
        "sql": """
            SELECT
                Amount,
                COUNT(*) AS Frequency
            FROM journal_entries
            GROUP BY Amount
            HAVING COUNT(*) > 5
            ORDER BY Frequency DESC;
        """
    },
    {
        "title": "Query 5: Entries with Missing or Blank Descriptions",
        "sql": """
            SELECT *
            FROM journal_entries
            WHERE Description IS NULL OR TRIM(Description) = '';
        """
    },
    {
       # 1. Define 'Write-off' and 'Suspense' as sensitive accounts.
# 2. Assume only 'user1' and 'user5' are authorized to APPROVE transactions in these accounts.
# 3. Find all entries in these accounts that were approved by anyone else (user2, user3, user4).

  "title": "Query 6: Unauthorized Approvals on Sensitive Accounts",
        "sql": """
            SELECT Entry_ID, Date, Account_Name, Amount, Created_By, Approved_By
            FROM journal_entries
            WHERE Account_Name IN ('Write-off', 'Suspense')
              AND Approved_By NOT IN ('user1', 'user5');
              """
    },
    {
        "title": "Query 7: Transactions with Suspicious Dates (Future or Very Old)",
        "sql": """
            SELECT *
            FROM journal_entries
            WHERE
                strptime(Date, '%d-%m-%Y') > CURRENT_DATE OR strptime(Date, '%d-%m-%Y') < '2020-01-01';
        """
    },
    {
        "title": "Query 8: Benford's Law Analysis (First Digit Distribution)",
        "sql": """
            SELECT
                SUBSTRING(CAST(Amount AS VARCHAR), 1, 1) AS FirstDigit,
                COUNT(*) AS Count,
                COUNT(*) * 100.0 / (SELECT COUNT(*) FROM journal_entries WHERE Amount > 0) AS Percentage
            FROM journal_entries
            WHERE Amount > 0
            GROUP BY FirstDigit
            ORDER BY FirstDigit;
        """
    },
    {
        "title": "Query 9: Analysis of Round Number Transactions",
        "sql": """
            SELECT *
            FROM journal_entries
            WHERE Amount % 100 = 0 AND Amount > 0
            ORDER BY Amount DESC;
        """
    },
    {
        "title": "Query 10: Critical Control Failure - Entries Approved by the Same User Who Created Them",
        "sql": """
            SELECT *
            FROM journal_entries
            WHERE Created_By = Approved_By;
        """
    }
]


for query in queries_to_run:
    print(f"--- {query['title']} ---")
    try:
        # Execute the query and fetch the result as a pandas DataFrame
        result_df = duckdb.query(query['sql']).to_df()
        
        # Display the result
        if result_df.empty:
            print("No results found for this query.")
        else:
            print(result_df)
            
    except Exception as e:
        print(f"An error occurred while running the query: {e}")
        
    print("\n" + "="*50 + "\n") # Separator for clarity



print("--- Final Analysis: Top 15 Entries with Highest Anomaly Scores ---")

flags_query = """
SELECT
    *,
    -- Flag for High Value
    (CASE WHEN Amount > 100000 THEN 1 ELSE 0 END) +
    -- Flag for Weekend Entry
    (CASE WHEN DAYOFWEEK(strptime(Date, '%d-%m-%Y')) IN (0, 6) THEN 1 ELSE 0 END) +
    -- Flag for Empty Description
    (CASE WHEN Description IS NULL OR TRIM(Description) = '' THEN 1 ELSE 0 END) +
    -- Flag for Round Number
    (CASE WHEN Amount % 100 = 0 AND Amount > 0 THEN 1 ELSE 0 END) +
    -- Flag for Self-Approval
    (CASE WHEN Created_By = Approved_By THEN 1 ELSE 0 END)
    AS AnomalyScore
FROM
    journal_entries;
"""

try:
    anomalies_df = duckdb.query(flags_query).to_df()
    sorted_anomalies = anomalies_df.sort_values(by='AnomalyScore', ascending=False)
    final_report_df = sorted_anomalies.reset_index(drop=True)
   
    print(final_report_df.head(15))

except Exception as e:
    print(f"An error occurred while calculating anomaly scores: {e}")

--- Data Loading ---
Successfully loaded CSV. Columns are: ['Entry_ID', 'Date', 'Time', 'Account_Name', 'Amount', 'Description', 'Created_By', 'Approved_By', 'Label_Anomaly']


--- Query 1: Top 10 Highest Value Transactions ---
   Entry_ID        Date      Time         Account_Name  Amount  \
0        43  06-05-2023  08:57:58                 Loan   49999   
1       183  15-03-2023  02:41:55                 Loan   49999   
2        45  30-06-2023  13:35:00                 Cash   49999   
3       188  11-06-2023  20:21:04             Suspense   49999   
4       189  18-06-2023  19:55:09             Suspense   49999   
5       179  27-01-2023  04:16:12              Revenue   49921   
6        30  13-01-2023  20:34:53                 Cash   49485   
7       196  04-04-2023  21:46:15  Accounts Receivable   49408   
8       104  06-02-2023  02:03:10                 Loan   49183   
9       126  02-03-2023  20:43:58             Suspense   49040   

        Description Created_By Approved_By  L